In [3]:
import os
import pandas as pd
import numpy as np
#!pip install crispr_chronos 
import chronos
import os
from taigapy import default_tc as tc
from taigapy import create_taiga_client_v3  
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()

In [4]:
chronos.__file__

'/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/__init__.py'

In [3]:
#!pip install google-cloud-storage

# Load RAS data

In [5]:
RASFullCounts = tc.get(name='rasscreen02062025-6bdb', file='RASFullCounts') 
RASFullGuideMap = tc.get(name='rasscreen02062025-6bdb', file='RASFullGuideMap')
RASSequenceQCReport = tc.get(name='rasscreen02020625-6bdb', file='RASSequenceQCReport')
#RASFullLfcSgrnaSeq = tc.download_to_cache(name='rasscreen02020625-6bdb', version=4, file='RASFullLfcSgrnaSeq')  # download_to_cache for raw
#RASFullNormCountsSgrnaSeq = tc.download_to_cache(name='rasscreen02062025-6bdb', version=4, file='RASFullNormCountsSgrnaSeq')  # download_to_cache for raw
#RASGuideAgreementScore = tc.get(name='rasscreen02062025-6bdb', version=4, file='RASGuideAgreementScore')
#RASRawCounts = tc.get(name='rasscreen02062025-6bdb', version=4, file='RASRawCounts')
#RASRawCountsAllpDNAWells = tc.get(name='rasscreen02062025-6bdb', version=4, file='RASRawCountsAllpDNAWells')
RASSequenceMap = tc.get(name='rasscreen02062025-6bdb', file='RASSequenceMap')



No dataset version provided. Using version 8.
No dataset version provided. Using version 8.
No dataset version provided. Using version 8.


[                      ]  0% |   0.0 s/B |   0.0 B /   1.4 KiB | ETA:  --:--:--
[##################]100% |   1.1 MiB/s |   1.4 KiB /   1.4 KiB | ETA:  00:00:00


No dataset version provided. Using version 8.


[                      ]  0% |   0.0 s/B |   0.0 B /   2.4 KiB | ETA:  --:--:--
[##################]100% |   2.2 MiB/s |   2.4 KiB /   2.4 KiB | ETA:  00:00:00


In [6]:
# Library file
from taigapy import TaigaClient
tc = TaigaClient()
RASCommonEssentialControls = tc.get(name='ras-library-files-35fa',file='RasCombinatoryCommonEssentialControls')
RASNonessentialControls = tc.get(name='ras-library-files-35fa', file='RasCombinatoryNonEssentialControls')



No dataset version provided. Using version 12.


[                      ]  0% |   0.0 s/B |   0.0 B /   1.7 KiB | ETA:  --:--:--
[##################]100% |   1.6 MiB/s |   1.7 KiB /   1.7 KiB | ETA:  00:00:00


No dataset version provided. Using version 12.


[                      ]  0% |   0.0 s/B |   0.0 B /   1.7 KiB | ETA:  --:--:--
[##################]100% |   1.7 MiB/s |   1.7 KiB /   1.7 KiB | ETA:  00:00:00


In [7]:
sorted(RASFullCounts.columns)

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'RAS-pDNA',
 'SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

# Read counts matrix

In [8]:
readcounts = RASFullCounts.copy()
readcounts = readcounts.fillna(0)

# Sequence map

In [11]:
chronos_sequence_map = RASSequenceMap[['SequenceName', 'Screen', 'Days']].copy()
chronos_sequence_map['pDNA_batch'] = 'RAS'
chronos_sequence_map = chronos_sequence_map.rename(
        columns={
            'SequenceName': 'sequence_ID', 
            'Screen': 'cell_line_name', 
            'Days': 'days'
        }
)

In [12]:
# chronos_sequence_map.iloc[0,2]=0
chronos_sequence_map['days']=chronos_sequence_map['days'].fillna(21) 
chronos_sequence_map

,sequence_ID,cell_line_name,days,pDNA_batch
0,SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),SNU1079_250x,25,RAS
1,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x,25,RAS
2,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410_125x,21,RAS
3,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x,21,RAS
4,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x,21,RAS
5,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x,21,RAS
6,COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),COLO205_250x,21,RAS
7,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),HPAFII_500x,21,RAS
8,BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_500x,21,RAS
9,BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_250x,21,RAS


In [13]:
sorted(chronos_sequence_map['sequence_ID'])

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'RAS-pDNA',
 'SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

# Quality controls for the raw counts matrix

## Remove black list cell line

In [15]:
RASSequenceQCReport = RASSequenceQCReport.drop(columns=['Unnamed: 0'])
RASSequenceQCReport


,SequenceName,NNMD,NNMD_zscore,AvgRepresentations,MedianCommonEssentials,MedianNonessentials,MeanCommonEssentials,MeanNonessentials
0,SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),-0.359873,-0.375076,1092.360131,-0.367533,-0.233138,-0.432999,-0.245638
1,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),-4.487811,-2.786580,744.848314,-0.832223,0.272179,-0.806931,0.230825
2,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),-6.439762,-3.763497,897.384976,-1.604345,0.155227,-1.486634,0.140744
3,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),-4.986809,-2.967967,498.497553,-0.762799,0.193320,-0.763068,0.196861
4,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),-8.620083,-5.523332,943.382625,-1.620380,0.300183,-1.651736,0.258015
5,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),-7.787617,-4.896570,577.830475,-1.555486,0.326217,-1.429382,0.290995
6,COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),-8.982244,-5.254199,855.656865,-1.274990,0.302333,-1.254965,0.316581
7,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),-6.905548,-4.734962,1177.173100,-1.732712,0.348497,-1.686927,0.359320
8,BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),-5.567603,-3.830898,1002.809323,-0.524419,0.196481,-0.532693,0.208939
9,BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),-4.505232,-3.342512,2727.901306,-0.599795,0.221996,-0.583475,0.225578


In [16]:
black_list = ['CUTO14', # Its non-essential distribution looks bad
              'SNU349_Day21_Parental',  # Its non-essential distribution looks bad
              'SNU349_Day25_Parental', # Its non-essential distribution looks bad
              "TUHR14TKB", # Its non-essential distribution looks bad
              'LU65_RepA_SEL00030', 'LU65_RepB_SEL00030', # Experimental error 
              'HCC2935_RepA', 'HCC2935_RepB',   #contaminated with 293Ts. 
              'NCIH2009_RepA', 'NCIH2009_RepB' # contaminated with 293Ts. 
              ] 
readcounts = readcounts.loc[:, ~readcounts.columns.isin(black_list)]
chronos_sequence_map = chronos_sequence_map.loc[~chronos_sequence_map['sequence_ID'].isin(black_list)]

In [17]:
# Remove low NNMD
black_list = RASSequenceQCReport.loc[RASSequenceQCReport['NNMD'] > -1.25, ].SequenceName.to_list()
black_list

['SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)']

In [18]:
chronos_sequence_map = chronos_sequence_map.loc[~chronos_sequence_map['sequence_ID'].isin(black_list)]
readcounts = readcounts.loc[:,chronos_sequence_map['sequence_ID'] ]


In [20]:
guide_map = RASFullGuideMap[['GuideLabel', 'GuideTargetSymbol']].copy()
guide_map = guide_map.rename(columns=({'GuideLabel': 'sgrna', 'GuideTargetSymbol': 'gene'}))
good_guides = RASFullGuideMap.loc[RASFullGuideMap['DropReasonsParalog'].isnull(),'GuideLabel']
guide_map = guide_map.loc[guide_map['sgrna'].isin(good_guides)]

In [21]:
readcounts = readcounts.loc[guide_map['sgrna']]


In [22]:
#--- Transpose read counts matrix into cell line by sgRNA
readcounts = readcounts.T


In [23]:
readcounts

,AAVS1_L5_ABCB7_R4,AAVS1_L5_ABCB7_R6,AAVS1_L5_ABCF1_R2,AAVS1_L5_ABCF1_R4,AAVS1_L5_ACTL7A_R2,AAVS1_L5_ACTL7A_R4,AAVS1_L5_ACTL7B_R4,AAVS1_L5_ACTL7B_R6,AAVS1_L5_ACTL9_R4,AAVS1_L5_ACTL9_R6,...,YWHAZ_L3_YAP1_R2,YWHAZ_L3_YAP1_R4,YWHAZ_L3_YWHAE_R4,YWHAZ_L3_YWHAE_R6,YWHAZ_L3_YWHAG_R4,YWHAZ_L3_YWHAG_R6,YY1_L1_AAVS1_R2,YY1_L5_AAVS1_R2,ZNF143_L1_AAVS1_R2,ZNF143_L3_AAVS1_R2
COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),321.0,515.0,238.0,410.0,2124.0,931.0,384.0,608.0,1110.0,2041.0,...,819.0,1742.0,4584.0,4445.0,1262.0,1066.0,126.0,54.0,1522.0,1039.0
SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),114.0,111.0,67.0,329.0,521.0,411.0,325.0,245.0,1158.0,898.0,...,862.0,947.0,1211.0,3091.0,689.0,691.0,91.0,901.0,426.0,820.0
BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),663.0,194.0,127.0,251.0,740.0,837.0,704.0,427.0,633.0,704.0,...,675.0,1143.0,1586.0,1809.0,924.0,436.0,239.0,331.0,976.0,1025.0
SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),0.0,1.0,70.0,135.0,939.0,319.0,454.0,376.0,339.0,164.0,...,776.0,456.0,1242.0,590.0,687.0,525.0,19.0,323.0,288.0,595.0
HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),79.0,34.0,54.0,185.0,1149.0,1058.0,912.0,478.0,598.0,688.0,...,1026.0,1843.0,1876.0,2093.0,921.0,654.0,140.0,80.0,1143.0,552.0
COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),177.0,138.0,224.0,145.0,958.0,288.0,506.0,428.0,461.0,413.0,...,275.0,457.0,621.0,497.0,183.0,183.0,182.0,110.0,872.0,427.0
HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),279.0,21.0,24.0,701.0,374.0,854.0,655.0,797.0,990.0,797.0,...,174.0,519.0,168.0,404.0,111.0,169.0,281.0,117.0,328.0,893.0
BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),431.0,428.0,336.0,491.0,1309.0,723.0,999.0,574.0,948.0,987.0,...,1254.0,2242.0,2901.0,2873.0,1107.0,938.0,243.0,532.0,888.0,1148.0
BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),440.0,263.0,221.0,165.0,1359.0,855.0,575.0,799.0,710.0,768.0,...,524.0,1379.0,1591.0,1770.0,1036.0,429.0,296.0,575.0,1219.0,862.0
SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),275.0,635.0,746.0,963.0,2721.0,2610.0,2183.0,1354.0,2878.0,1895.0,...,2322.0,6963.0,8047.0,7390.0,3993.0,3327.0,525.0,1017.0,1090.0,1667.0


In [24]:
sorted(readcounts.index)

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'RAS-pDNA',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

# Chronos training for parental

In [26]:
index_safeharbor_sgrna = guide_map['gene'].isin(['AAVS1', 'chr2'])
index_hart_nonessential_sgrna = guide_map['gene'].isin(RASNonessentialControls['GuideTargetSymbol'])


In [27]:
import chronos
chronos.__file__

'/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/__init__.py'

In [28]:
#--- Remove outgrowths by chronos
chronos.nan_outgrowths(readcounts,chronos_sequence_map, guide_map)

calculating LFC
finding maximum LFC calls


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:418: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  max_lfc = lfc.groupby(ggtemp, axis=1).max()
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:433: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  max_row_2nd_column = lfc_filtered.T.groupby(ggtemp, axis=0).agg(second_highest).T


filtering
finding second highest LFC calls
finding sequences and guides with outgrowth


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:444: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  masked = masked.reindex(index=lfc_filtered.index, columns=ggtemp.values).fillna(False)
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:449: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.reindex(index=readcounts.index, columns=readcounts.columns).fillna(False).astype(bool)


NAing 202 readcounts (0.00015 of total)


In [29]:
#--- It is much better to use cutting than noncutting controls, and as many as possible.
negative_control_sgrnas = guide_map.loc[index_hart_nonessential_sgrna,]
negative_control_sgrnas = negative_control_sgrnas['sgrna'].to_list()


In [30]:
chronos_sequence_map['days']=[int(i) for i in chronos_sequence_map['days']]
chronos_sequence_map

,sequence_ID,cell_line_name,days,pDNA_batch
1,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x,25,RAS
2,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410_125x,21,RAS
3,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x,21,RAS
4,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x,21,RAS
5,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x,21,RAS
6,COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),COLO205_250x,21,RAS
7,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),HPAFII_500x,21,RAS
8,BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_500x,21,RAS
9,BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_250x,21,RAS
10,SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),SNU410_500x,21,RAS


In [31]:
guide_map

,sgrna,gene
0,AAVS1_L5_ABCB7_R4,ABCB7
1,AAVS1_L5_ABCB7_R6,ABCB7
2,AAVS1_L5_ABCF1_R2,ABCF1
3,AAVS1_L5_ABCF1_R4,ABCF1
6,AAVS1_L5_ACTL7A_R2,ACTL7A
...,...,...
84195,YWHAZ_L3_YWHAG_R6,YWHAZ_YWHAG
84196,YY1_L1_AAVS1_R2,YY1
84197,YY1_L5_AAVS1_R2,YY1
84198,ZNF143_L1_AAVS1_R2,ZNF143


In [32]:
negative_control_sgrnas

['AAVS1_L5_ACTL7A_R2',
 'AAVS1_L5_ACTL7A_R4',
 'AAVS1_L5_ACTL7B_R4',
 'AAVS1_L5_ACTL7B_R6',
 'AAVS1_L5_ACTL9_R4',
 'AAVS1_L5_ACTL9_R6',
 'AAVS1_L5_ADAM18_R2',
 'AAVS1_L5_ADAM18_R6',
 'AAVS1_L5_ADAM20_R2',
 'AAVS1_L5_ADAM20_R6',
 'AAVS1_L5_ADAM30_R4',
 'AAVS1_L5_ADAM30_R6',
 'AAVS1_L5_AICDA_R2',
 'AAVS1_L5_ALPG_R2',
 'AAVS1_L5_ALPG_R6',
 'AAVS1_L5_ALX3_R6',
 'AAVS1_L5_ASB17_R4',
 'AAVS1_L5_ASB17_R6',
 'AAVS1_L5_ATP4B_R4',
 'AAVS1_L5_ATP4B_R6',
 'AAVS1_L5_ATP6V1G3_R2',
 'AAVS1_L5_ATP6V1G3_R4',
 'AAVS1_L5_B3GNT6_R4',
 'AAVS1_L5_B3GNT6_R6',
 'AAVS1_L5_BPIFA3_R2',
 'AAVS1_L5_CABP2_R2',
 'AAVS1_L5_CABP2_R4',
 'AAVS1_L5_CABP5_R2',
 'AAVS1_L5_CABP5_R4',
 'AAVS1_L5_CCT8L2_R4',
 'AAVS1_L5_CCT8L2_R6',
 'AAVS1_L5_CEACAM7_R2',
 'AAVS1_L5_CELA2A_R2',
 'AAVS1_L5_CELA2A_R4',
 'AAVS1_L5_CELA3A_R4',
 'AAVS1_L5_CELA3B_R2',
 'AAVS1_L5_CELA3B_R4',
 'AAVS1_L5_CFHR5_R4',
 'AAVS1_L5_CFHR5_R6',
 'AAVS1_L5_CHAT_R8',
 'AAVS1_L5_CLCA1_R4',
 'AAVS1_L5_CNBD1_R2',
 'AAVS1_L5_CNBD1_R6',
 'AAVS1_L5_CSHL1_R2',
 'AAVS1_

In [33]:
readcounts

,AAVS1_L5_ABCB7_R4,AAVS1_L5_ABCB7_R6,AAVS1_L5_ABCF1_R2,AAVS1_L5_ABCF1_R4,AAVS1_L5_ACTL7A_R2,AAVS1_L5_ACTL7A_R4,AAVS1_L5_ACTL7B_R4,AAVS1_L5_ACTL7B_R6,AAVS1_L5_ACTL9_R4,AAVS1_L5_ACTL9_R6,...,YWHAZ_L3_YAP1_R2,YWHAZ_L3_YAP1_R4,YWHAZ_L3_YWHAE_R4,YWHAZ_L3_YWHAE_R6,YWHAZ_L3_YWHAG_R4,YWHAZ_L3_YWHAG_R6,YY1_L1_AAVS1_R2,YY1_L5_AAVS1_R2,ZNF143_L1_AAVS1_R2,ZNF143_L3_AAVS1_R2
COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),321.0,515.0,238.0,410.0,2124.0,931.0,384.0,608.0,1110.0,2041.0,...,819.0,1742.0,4584.0,4445.0,1262.0,1066.0,126.0,54.0,1522.0,1039.0
SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),114.0,111.0,67.0,329.0,521.0,411.0,325.0,245.0,1158.0,898.0,...,862.0,947.0,1211.0,3091.0,689.0,691.0,91.0,901.0,426.0,820.0
BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),663.0,194.0,127.0,251.0,740.0,837.0,704.0,427.0,633.0,704.0,...,675.0,1143.0,1586.0,1809.0,924.0,436.0,239.0,331.0,976.0,1025.0
SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),0.0,1.0,70.0,135.0,939.0,319.0,454.0,376.0,339.0,164.0,...,776.0,456.0,1242.0,590.0,687.0,525.0,19.0,323.0,288.0,595.0
HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),79.0,34.0,54.0,185.0,1149.0,1058.0,912.0,478.0,598.0,688.0,...,1026.0,1843.0,1876.0,2093.0,921.0,654.0,140.0,80.0,1143.0,552.0
COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),177.0,138.0,224.0,145.0,958.0,288.0,506.0,428.0,461.0,413.0,...,275.0,457.0,621.0,497.0,183.0,183.0,182.0,110.0,872.0,427.0
HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),279.0,21.0,24.0,701.0,374.0,854.0,655.0,797.0,990.0,797.0,...,174.0,519.0,168.0,404.0,111.0,169.0,281.0,117.0,328.0,893.0
BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),431.0,428.0,336.0,491.0,1309.0,723.0,999.0,574.0,948.0,987.0,...,1254.0,2242.0,2901.0,2873.0,1107.0,938.0,243.0,532.0,888.0,1148.0
BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),440.0,263.0,221.0,165.0,1359.0,855.0,575.0,799.0,710.0,768.0,...,524.0,1379.0,1591.0,1770.0,1036.0,429.0,296.0,575.0,1219.0,862.0
SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),275.0,635.0,746.0,963.0,2721.0,2610.0,2183.0,1354.0,2878.0,1895.0,...,2322.0,6963.0,8047.0,7390.0,3993.0,3327.0,525.0,1017.0,1090.0,1667.0


In [34]:

model = chronos.Chronos(
    sequence_map={"RASV2": chronos_sequence_map},
    guide_gene_map={"RASV2": guide_map},
    readcounts={"RASV2": readcounts},
    negative_control_sgrnas={'RASV2': negative_control_sgrnas}
)

# model = chronos.Chronos(
#     sequence_map={"RASV2": chronos_sequence_map},
#     guide_gene_map={"RASV2": guide_map},
#     readcounts={"RASV2": readcounts},
#     negative_control_sgrnas={'RASV2': negative_control_sgrnas},
#     to_normalize_readcounts=True
# )



normalizing readcounts


Finding all unique guides and genes
found 83999 unique guides and 21046 unique genes in RASV2
found 83999 unique guides and 21046 unique genes overall

finding guide-gene mapping indices

finding all unique sequenced replicates, cell lines, and pDNA batches
found 15 unique sequences (excluding pDNA) and 15 unique cell lines in RASV2
found 15 unique replicates and 15 unique cell lines overall

finding replicate-cell line mappings indices

finding replicate-pDNA mappings indices


assigning float constants
Estimating or aligning variances
	Estimating excess variance (alpha) for RASV2
Between 0 (batch=Index(['RASV2_RAS'], dtype='object')) and 0 (batch=Index(['RASV2_RAS'], dtype='object')) negative control sgRNAs were found to be systematically over- or under-represented in the screens and excluded.
Creating excess variance tensors
	Created excess variance tensor for RASV2 with shape [15, 1]
initializing graph

building gene effect mask

building doubling vectors
m

2025-02-07 18:26:54.925181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:1196: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  key: readcounts[key]\
2025-02-07 18:26:55.678302: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


	built normalized timepoints for RASV2 with shape (15, 83999) (replicates X guides)

building t0 reads


Building variables

building t0 reads estimate
made t0 batch with shape (1, 83999) for RASV2
building guide efficacy
built guide efficacy: shape [1, 84000]
building growth rate
built growth rate: shape {'RASV2': [15, 1]}

building cell line efficacy
built cell line efficacy: shapes {'RASV2': [15, 1]}
building screen delay
built screen delay
building gene effect
built core gene effect: 15 cell lines by 21046 genes


Connecting graph nodes in model

building effective days
built effective days, shapes {'RASV2': [15, 21046]}

building gene effect growth graph nodes
built gene effect growth graph nodes, shapes {'RASV2': [15, 21046]}

building combined efficacy
built combined efficacy, shape {'RASV2': [15, 83999]}
built expanded combined efficacy, shapes {'RASV2': [15, 83999]}

building growth estimates of edited cells and overall estimates of fold change in guide abundance
built growth 

2025-02-07 18:27:00.266457: W tensorflow/c/c_api.cc:291] Operation '{name:'excess_variance/RASV2/Assign' id:6 op device:{requested: '', assigned: ''} def:{{{node excess_variance/RASV2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_DOUBLE, validate_shape=false](excess_variance/RASV2, excess_variance/RASV2/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


estimating initial screen efficacy and gene effect
	 RASV2


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:1772: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  mean_fold_change = fold_change\


	verifying graph integrity
verifying user inputs
verifying variables
verifying calculated terms
	RASV2 _gene_effect
	RASV2 _selected_efficacies
	RASV2_predicted_readcounts_unscaled
	RASV2 _predicted_readcounts
	RASV2 _normalized_readcounts
	RASV2 _cost_presum
sess run
	RASV2 _cost
	RASV2 _full_costs
ready to train


In [35]:
model.train(1001, report_freq=50, burn_in_period=50, ge_only=0)

NB2 cost 0.23868135951167435
Full cost 0.28208200593705557
relative_growth_rate
	RASV2 max 1.041, min 0.96700
mean guide efficacy 0.992066658285559
t0_offset SD: [('RASV2', 6.334490229588964e-05)]

gene mean -0.2357296529930946
SD of gene means 0.333472397495349
Mean of gene SDs 0.3156867725245078



51 epochs trained, time taken 0:01:25, projected remaining 0:26:30
NB2 cost 0.1692102584432443
Full cost 0.20434136211406073
relative_growth_rate
	RASV2 max 1.251, min 0.75559
mean guide efficacy 0.9405208335310217
t0_offset SD: [('RASV2', 0.1262690456968865)]

gene mean -0.058292717680973515
SD of gene means 0.4264617045673797
Mean of gene SDs 0.334100431031552



101 epochs trained, time taken 0:02:50, projected remaining 0:25:11
NB2 cost 0.1630353479267856
Full cost 0.19016697294460194
relative_growth_rate
	RASV2 max 1.261, min 0.77449
mean guide efficacy 0.9095618098200885
t0_offset SD: [('RASV2', 0.10205982001523602)]

gene mean 0.0681372271848156
SD of gene means 0.42342323406321786


In [36]:
model

Chronos model with libraries ['RASV2'], 15 total cell lines, 21046 total genes, trained for 1001 epochs

In [37]:
BASEDIR = "/home/ubuntu/paralog/ras_screens/analysis/RASScreen02062025/"
model.save( os.path.join(BASEDIR, "0206_chronos"), overwrite=True)

In [33]:
#model = chronos.load_saved_model(os.path.join(BASEDIR, "chronos"))

In [34]:
# model

Chronos model with libraries ['ParalogV2'], 259 total cell lines, 12196 total genes, trained for 1001 epochs

In [38]:
gene_effect = model.gene_effect

# Copy number correction

In [44]:
from taigapy import create_taiga_client_v3  
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()

In [46]:
#OmicsCNGeneFullProfileRASV2 = chronos.read_hdf5("../data/RASScreen05132024/OmicsCNGeneFullProfileRASV2.hdf5")
OmicsCNGeneFullProfileRASV2 = tc.get(name='rasscreen02062025-6bdb', file='OmicsCNGeneFullProfileRASV2')  # download_to_cache for raw
OmicsCNGeneFullProfileRASV2 = OmicsCNGeneFullProfileRASV2[gene_effect.columns]

RASScreenMap = RASSequenceMap[['Screen', 'CellLine', 'ModelID']].drop_duplicates().set_index('Screen')
RASScreenMap = RASScreenMap.loc[gene_effect.index]
RASScreenMap.loc[RASScreenMap['ModelID'].isna(), 'ModelID'] = RASScreenMap.loc[RASScreenMap['ModelID'].isna(), ].index



No dataset version provided. Using version 9.


In [47]:
OmicsCNGeneFullProfileRASV2

,ABCB7,ABCF1,ACTL7A,ACTL7B,ACTL9,ADAM18,ADAM20,ADAM30,AFG3L2,AHCY,...,YWHAZ_TEAD4,YWHAZ_TIAM1,YWHAZ_TSC1,YWHAZ_TSC2,YWHAZ_WWTR1,YWHAZ_YAP1,YWHAZ_YWHAE,YWHAZ_YWHAG,YY1,ZNF143
ACH-000628,0.773267,1.363993,1.059823,1.059823,0.964199,0.731485,1.303075,0.700178,1.299878,2.090883,...,2.039647,2.572728,2.318347,2.551699,2.835871,2.536641,2.329836,2.742059,1.303075,1.115954
ACH-002999,0.528507,0.950967,1.042014,1.042014,0.752135,0.526512,1.040944,1.692817,1.021102,1.035879,...,2.124644,1.591220,2.105306,2.101855,2.684518,2.134728,1.904631,2.123021,1.032738,1.057404
ACH-001394,1.404696,0.968820,1.425395,1.425395,0.643974,1.045293,0.998995,1.088390,1.454167,1.413716,...,3.269951,2.660348,3.186200,2.555889,3.565709,2.693563,2.591325,3.189423,0.972168,0.712122
ACH-001973,0.493707,1.003962,0.979017,0.979017,0.933863,1.008685,0.975487,1.501060,1.000903,1.002260,...,1.999872,1.993012,1.987702,1.983109,2.014877,1.980276,1.494177,2.012780,0.975487,0.875413
ACH-001163,0.951227,0.995334,0.976192,0.976192,0.955288,1.296919,0.987842,0.998874,1.003802,1.001033,...,2.282126,2.259918,2.265028,2.284794,2.264148,2.273092,2.288722,2.291078,0.987842,0.984256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001039,0.547492,0.985400,1.150980,1.150980,0.939516,0.589237,0.970185,0.624576,0.630750,1.457635,...,2.061208,2.044925,2.274937,2.090514,2.351860,2.011607,2.175655,2.060817,0.970185,0.935269
ACH-003477,0.651117,1.067069,1.465494,1.465494,0.713540,0.727090,0.725727,1.423087,1.096629,1.426912,...,1.792136,2.174973,2.485774,1.816427,2.173581,2.168456,1.805425,2.102617,0.725727,1.420103
ACH-001971,0.956893,1.244266,0.614354,0.614354,0.913798,1.108575,1.134821,0.859132,0.860810,1.212081,...,2.236181,1.992968,1.722929,2.039348,1.964923,1.981818,1.960712,3.276812,1.134821,0.912347
ACH-002761,0.625845,1.046931,1.381593,1.381593,0.994020,1.049747,0.704922,0.724560,1.040793,1.374272,...,2.413407,2.108323,2.415335,2.102967,1.764992,1.797894,1.748584,1.774282,0.704922,1.046329


In [48]:
copy_number_profile = ( 
    RASScreenMap.
        merge(OmicsCNGeneFullProfileRASV2, how = 'left', left_on = 'ModelID', right_index=True).
        drop(['CellLine'], axis=1).
        drop(['ModelID'], axis=1)
)

In [49]:
copy_number_profile

,ABCB7,ABCF1,ACTL7A,ACTL7B,ACTL9,ADAM18,ADAM20,ADAM30,AFG3L2,AHCY,...,YWHAZ_TEAD4,YWHAZ_TIAM1,YWHAZ_TSC1,YWHAZ_TSC2,YWHAZ_WWTR1,YWHAZ_YAP1,YWHAZ_YWHAE,YWHAZ_YWHAG,YY1,ZNF143
cell_line_name,,,,,,,,,,,,,,,,,,,,,
BXPC3_125x,0.929710,0.870387,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,1.786170,...,2.391036,2.165055,2.511888,2.480956,2.645840,2.781340,2.094268,2.669955,1.201395,0.922648
BXPC3_250x,0.929710,0.870387,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,1.786170,...,2.391036,2.165055,2.511888,2.480956,2.645840,2.781340,2.094268,2.669955,1.201395,0.922648
BXPC3_500x,0.929710,0.870387,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,1.786170,...,2.391036,2.165055,2.511888,2.480956,2.645840,2.781340,2.094268,2.669955,1.201395,0.922648
COLO205_250x,0.547492,0.985400,1.150980,1.150980,0.939516,0.589237,0.970185,0.624576,0.630750,1.457635,...,2.061208,2.044925,2.274937,2.090514,2.351860,2.011607,2.175655,2.060817,0.970185,0.935269
COLO678_250x,0.814662,0.885113,1.043124,1.043124,0.888962,0.965243,0.930270,1.016781,0.974481,1.826741,...,2.357654,1.959277,1.888079,1.855987,2.455416,3.015285,1.869864,2.052173,0.917738,0.925982
HCT15_250x,0.488206,0.995882,0.994496,0.994496,0.958509,0.992196,1.011965,0.998813,0.997424,1.002760,...,1.993548,1.982308,1.986692,1.984069,1.987026,1.985410,1.989876,1.996270,0.981362,0.993214
HPAFII_500x,0.766498,0.900468,0.879374,0.879374,0.916412,1.010747,1.568399,1.029765,0.859138,1.579669,...,1.812036,1.864389,1.913173,1.614179,2.320285,2.220552,2.066054,2.337010,1.568399,0.790848
IPC298_250x,0.647532,1.113070,0.949177,0.949177,1.015841,1.171571,0.860844,3.368852,1.567224,1.205413,...,2.263725,2.520030,2.020036,2.000134,1.797210,2.154006,1.722619,2.730205,0.854500,0.930137
IPC298_500x,0.647532,1.113070,0.949177,0.949177,1.015841,1.171571,0.860844,3.368852,1.567224,1.205413,...,2.263725,2.520030,2.020036,2.000134,1.797210,2.154006,1.722619,2.730205,0.854500,0.930137


In [50]:
sorted(copy_number_profile.index)

['BXPC3_125x',
 'BXPC3_250x',
 'BXPC3_500x',
 'COLO205_250x',
 'COLO678_250x',
 'HCT15_250x',
 'HPAFII_500x',
 'IPC298_250x',
 'IPC298_500x',
 'PANC0203',
 'PANC1',
 'SNU410_125x',
 'SNU410_250x',
 'SNU410_500x',
 'SNU478_250x']

In [51]:
copy_number_profile = copy_number_profile.fillna(1)
copy_number_profile = copy_number_profile.loc[gene_effect.index, gene_effect.columns ]
out = chronos.alternate_CN(gene_effect, copy_number_profile)



Fitting cell line group 1 of 1
finding low CN gene effect shifts
smoothing and interpolating cutting toxicity for all genes
constructed spline matrix of shape 315690, 105
	cost: 0.1373560570732321
	cost: 0.1294106828950655
	cost: 0.12707387338734488
	cost: 0.12619295510709275
	cost: 0.1257891634571574
	cost: 0.1255448763039351
generating matrix


In [52]:
gene_effect_target_corrected = out[0]
shifts_target = out[1]

In [53]:
gene_effect_final = gene_effect_target_corrected.copy()

In [44]:
#gene_effect.columns  = RASTargetMap['GuideTargetSymbol']
gene_effect_final -= gene_effect_final.reindex(columns=RASNonessentialControls['GuideTargetSymbol'].to_list()).median(axis=1).median()
gene_effect_final /= gene_effect_final.reindex(columns= RASCommonEssentialControls['GuideTargetSymbol'].to_list()).median(axis=1).abs().median()



In [55]:
chronos.write_hdf5(gene_effect_final, "0206RASGeneEffect.hdf5")
chronos.write_hdf5(gene_effect, "0206RASGeneEffectUncorrected.hdf5")

# Uploading chronos gene effect

In [56]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset('rasscreen02062025-6bdb', 
                            "Uploading Chronos Gene Effect and the copy number profile for chronos correction", 
                            additions=[
    UploadedFile('RASGeneEffectUncorrected', 
                 local_path=os.path.join("0206RASGeneEffectUncorrected.hdf5"), 
                 format=LocalFormat.HDF5_MATRIX),
     UploadedFile('RASGeneEffect', 
                 local_path=os.path.join("0206RASGeneEffect.hdf5"), 
                 format=LocalFormat.HDF5_MATRIX)
    ]
)

Created temporary upload session 51076450cb5d497c911bbc27dfc298cc
Uploading 0206RASGeneEffectUncorrected.hdf5 to S3
Finished uploading 0206RASGeneEffectUncorrected.hdf5 to S3
Completed uploading 0206RASGeneEffectUncorrected.hdf5 to S3
Added 0206RASGeneEffectUncorrected.hdf5 to upload session 51076450cb5d497c911bbc27dfc298cc
Uploading 0206RASGeneEffect.hdf5 to S3
Finished uploading 0206RASGeneEffect.hdf5 to S3
Completed uploading 0206RASGeneEffect.hdf5 to S3
Added 0206RASGeneEffect.hdf5 to upload session 51076450cb5d497c911bbc27dfc298cc
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen02062025-6bdb/10



In [57]:
RASGeneEffect = tc.get(name='rasscreen02062025-6bdb',file='RASGeneEffect')  # download_to_cache for raw


No dataset version provided. Using version 10.


[                      ]  0% |   0.0 s/B |   0.0 B /   2.5 MiB | ETA:  --:--:--
[##################]100% |  83.0 MiB/s |   2.5 MiB /   2.5 MiB | Time:  0:00:00


# Create ScreenMap

In [75]:
RASScreenMap = tc.get(name='rasscreen02062025-6bdb',file='RASScreenMap')


No dataset version provided. Using version 11.


In [76]:
RASScreenMap.columns

Index(['Screen', 'CellLine', 'ScreenType', 'Treatment', 'Source', 'Cas',
       'NNMD', 'NNMD_zscore', 'MedianCommonEssentials', 'MedianNonessentials',
       'MeanCommonEssentials', 'MeanNonessentials', 'Days', 'ModelID',
       'PatientID', 'CellLineName', 'StrippedCellLineName', 'CCLEName',
       'DepmapModelType', 'OncotreeLineage', 'OncotreePrimaryDisease',
       'OncotreeSubtype', 'OncotreeCode', 'SangerModelID', 'COSMICID',
       'CRISPR', 'CopyNumber', 'Expression', 'LOH', 'SpearmanAVANA',
       'UsedInChronos'],
      dtype='object')

In [77]:
RASScreenMap['UsedInChronos'] = RASScreenMap['Screen'].isin(RASGeneEffect.index)

In [78]:
RASScreenMap.to_csv("RASScreenMap0206.csv", index=False)

## Uploading screenmap

In [62]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset('rasscreen02062025-6bdb', 
                            "Uploading RASScreenMap", 
                            additions=[
    UploadedFile('RASScreenMap', 
                 local_path=os.path.join("RASScreenMap0206.csv"), 
                 format=LocalFormat.CSV_TABLE)
    ]
)

Created temporary upload session e346937548834fa4a95bb7c19a762c74
Uploading RASScreenMap0206.csv to S3
Finished uploading RASScreenMap0206.csv to S3
Completed uploading RASScreenMap0206.csv to S3
Added RASScreenMap0206.csv to upload session e346937548834fa4a95bb7c19a762c74
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen02062025-6bdb/11



# Split Parental screens

In [79]:
RASGeneEffect = tc.get(name='rasscreen02062025-6bdb', file='RASGeneEffect')  
RASScreenMap = tc.get(name='rasscreen02062025-6bdb', file='RASScreenMap')

No dataset version provided. Using version 11.
No dataset version provided. Using version 11.


In [80]:
RASScreenMap

,Screen,CellLine,ScreenType,Treatment,Source,Cas,NNMD,NNMD_zscore,MedianCommonEssentials,MedianNonessentials,...,OncotreeSubtype,OncotreeCode,SangerModelID,COSMICID,CRISPR,CopyNumber,Expression,LOH,SpearmanAVANA,UsedInChronos
0,SNU1079_250x,SNU1079,Parental,250x,Sellers,Cas9VY,-0.359873,-0.375076,-1.443211,0.197779,...,Intrahepatic Cholangiocarcinoma,IHCH,SIDM00164,NaN,True,True,True,True,0.271326,False
1,COLO678_250x,COLO678,Parental,250x,Sellers,Cas9VY,-4.487811,-2.786580,-1.620380,0.300183,...,Colon Adenocarcinoma,COAD,SIDM00957,910689.0,True,True,True,True,0.726015,True
2,SNU410_125x,SNU410,Parental,125x,Sellers,Cas9VY,-6.439762,-3.763497,-2.257279,0.135775,...,Pancreatic Adenocarcinoma,PAAD,None,NaN,True,True,True,False,0.739271,True
3,BXPC3_125x,BXPC3,Parental,125x,Sellers,Cas9VY,-4.986809,-2.967967,-0.367533,-0.233138,...,Pancreatic Adenocarcinoma,PAAD,SIDM00132,906693.0,True,True,True,True,0.689755,True
4,SNU478_250x,SNU478,Parental,250x,Sellers,Cas9VY,-8.620083,-5.523332,-0.879347,0.331256,...,Ampullary Carcinoma,AMPCA,SIDM00160,NaN,True,True,True,True,0.696835,True
5,HCT15_250x,HCT15,Parental,250x,Sellers,Cas9VY,-7.787617,-4.896570,-1.555486,0.326217,...,Colon Adenocarcinoma,COAD,SIDM00789,905937.0,True,True,True,True,NaN,True
6,COLO205_250x,COLO205,Parental,250x,Sellers,Cas9VY,-8.982244,-5.254199,-0.762799,0.193320,...,Colon Adenocarcinoma,COAD,SIDM00826,905961.0,True,True,False,True,NaN,True
7,HPAFII_500x,HPAFII,Parental,500x,Sellers,Cas9VY,-6.905548,-4.734962,-1.274990,0.302333,...,Pancreatic Adenocarcinoma,PAAD,SIDM00669,724869.0,True,True,True,True,NaN,True
8,BXPC3_500x,BXPC3,Parental,500x,Sellers,Cas9VY,-5.567603,-3.830898,-1.604345,0.155227,...,Pancreatic Adenocarcinoma,PAAD,SIDM00132,906693.0,True,True,True,True,0.751704,True
9,BXPC3_250x,BXPC3,Parental,250x,Sellers,Cas9VY,-4.505232,-3.342512,-0.832223,0.272179,...,Pancreatic Adenocarcinoma,PAAD,SIDM00132,906693.0,True,True,True,True,0.698293,True


In [81]:
RASScreenMapFiltered =  RASScreenMap.loc[RASScreenMap['UsedInChronos'] & (RASScreenMap['ScreenType'] == 'Parental'),]
RASScreenMapFiltered = RASScreenMapFiltered.loc[RASScreenMapFiltered['Screen'] != 'NCIH747.WRS00022']

In [82]:
RASGeneEffectParental = RASGeneEffect.loc[RASScreenMapFiltered['Screen']]

In [83]:
RASGeneEffectParental.index = RASScreenMapFiltered['ModelID']

In [84]:
RASGeneEffectParental = RASGeneEffectParental.T


In [85]:
RASGeneEffectParental=RASGeneEffectParental.loc[:, RASGeneEffectParental.columns.notnull()]
RASGeneEffectParental #alina added. Is this meant to be?

ModelID,ACH-000350,ACH-000517,ACH-000535,ACH-000377,ACH-000997,ACH-001039,ACH-000094,ACH-000535,ACH-000535,ACH-000517,ACH-000517,ACH-000915,ACH-000915,ACH-000164,ACH-000042
ABCB7,-1.026165,-1.269300,-1.384303,-2.832365,-2.121288,-0.686798,-1.154172,-0.937102,-1.532415,-1.009241,-1.544603,-1.007793,-1.806643,-0.919327,-1.235301
ABCF1,-0.542321,-0.704972,-1.686040,-0.994261,-1.722249,-0.556643,-0.704100,-1.008067,-0.998674,-0.845981,-1.351780,-0.512084,-0.956974,-0.409608,-0.681165
ACTL7A,0.081554,-0.326424,-0.063443,-0.037090,0.087091,0.099310,-0.084590,-0.066941,0.003357,0.035917,-0.006383,0.114998,-0.552725,0.054804,0.129136
ACTL7B,-0.233511,-0.138187,-0.128745,0.016470,0.067094,-0.044075,0.027178,0.019151,0.082310,-0.024235,-0.127424,-0.011486,0.135381,-0.178976,0.173921
ACTL9,0.264810,0.065104,-0.110494,-0.253986,-0.096154,0.018432,0.137511,0.101906,-0.064035,-0.063625,0.053511,-0.127010,-0.173856,-0.043117,0.115863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAZ_YAP1,-0.090906,-0.021487,-0.391432,0.030264,-0.072737,-0.608993,-0.721334,0.028433,-0.169789,-0.069597,-0.040199,-0.320981,-0.345279,-0.414866,-0.004657
YWHAZ_YWHAE,0.415166,0.057780,-0.021350,0.104784,0.102618,-0.488731,-1.081233,0.280521,-0.084897,0.114550,-0.003847,-0.849533,-1.281949,-0.007738,0.200468
YWHAZ_YWHAG,0.138467,0.105639,-0.140157,-0.039551,-0.384428,-0.914749,-1.261066,-0.216320,-0.635761,0.277790,0.333026,-0.842936,-0.243646,-0.465336,-0.013153
YY1,-1.592961,-0.437047,-1.223894,-0.904607,-1.185678,-0.666054,-0.802432,-1.195769,-0.935734,-0.696037,-0.614328,-0.781930,-0.327797,-1.388582,-0.647910


In [86]:
RASGeneEffectParental=RASGeneEffectParental.loc[:,~RASGeneEffectParental.columns.duplicated()].copy()

In [87]:
RASGeneEffectParental.to_csv("0206RASGeneEffectParentalByModelID.csv")

In [88]:
RASGeneEffectParental

ModelID,ACH-000350,ACH-000517,ACH-000535,ACH-000377,ACH-000997,ACH-001039,ACH-000094,ACH-000915,ACH-000164,ACH-000042
ABCB7,-1.026165,-1.269300,-1.384303,-2.832365,-2.121288,-0.686798,-1.154172,-1.007793,-0.919327,-1.235301
ABCF1,-0.542321,-0.704972,-1.686040,-0.994261,-1.722249,-0.556643,-0.704100,-0.512084,-0.409608,-0.681165
ACTL7A,0.081554,-0.326424,-0.063443,-0.037090,0.087091,0.099310,-0.084590,0.114998,0.054804,0.129136
ACTL7B,-0.233511,-0.138187,-0.128745,0.016470,0.067094,-0.044075,0.027178,-0.011486,-0.178976,0.173921
ACTL9,0.264810,0.065104,-0.110494,-0.253986,-0.096154,0.018432,0.137511,-0.127010,-0.043117,0.115863
...,...,...,...,...,...,...,...,...,...,...
YWHAZ_YAP1,-0.090906,-0.021487,-0.391432,0.030264,-0.072737,-0.608993,-0.721334,-0.320981,-0.414866,-0.004657
YWHAZ_YWHAE,0.415166,0.057780,-0.021350,0.104784,0.102618,-0.488731,-1.081233,-0.849533,-0.007738,0.200468
YWHAZ_YWHAG,0.138467,0.105639,-0.140157,-0.039551,-0.384428,-0.914749,-1.261066,-0.842936,-0.465336,-0.013153
YY1,-1.592961,-0.437047,-1.223894,-0.904607,-1.185678,-0.666054,-0.802432,-0.781930,-1.388582,-0.647910


In [90]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset('rasscreen02062025-6bdb', 
                            "Uploading RASGeneEffectParentalByModelID", 
                            additions=[
    UploadedFile('RASGeneEffectParentalByModelID', 
                 local_path=os.path.join("0206RASGeneEffectParentalByModelID.csv"), 
                 format=LocalFormat.CSV_MATRIX)
    ]
)

Created temporary upload session 40352f6b8cea42e788b5902dd7505583
Uploading 0206RASGeneEffectParentalByModelID.csv to S3
Finished uploading 0206RASGeneEffectParentalByModelID.csv to S3
Completed uploading 0206RASGeneEffectParentalByModelID.csv to S3
Added 0206RASGeneEffectParentalByModelID.csv to upload session 40352f6b8cea42e788b5902dd7505583
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen02062025-6bdb/12

